<a href="https://colab.research.google.com/github/eliaslimmer/PraktikumMushroom/blob/main/data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Requirements Installation



In [ ]:
!pip install transformers datasets evaluate accelerate
!pip install --upgrade datasets
!pip install scipy
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
from google.colab import drive
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset
import torch.nn.functional as F
import gc
import json
from tqdm import tqdm
from scipy.stats import spearmanr #For the spearman correlation computation

# Get helper functions from github


In [ ]:
# Clone the GitHub repository
!git clone https://github.com/eliaslimmer/PraktikumMushroom.git

# Change directory to the cloned repository
%cd PraktikumMushroom

import src.baseline_model as baseline_model
import src.data_preparation as data_preparation
import src.helpers as helpers
import src.metrics as metrics
!ls

Cloning into 'PraktikumMushroom'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 88 (delta 46), reused 56 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (88/88), 5.16 MiB | 10.28 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/PraktikumMushroom/PraktikumMushroom


ModuleNotFoundError: No module named 'baseline_model'

In [4]:
!git pull origin main

From https://github.com/eliaslimmer/PraktikumMushroom
 * branch            main       -> FETCH_HEAD
Already up to date.


#Login to huggingface

In [5]:
from huggingface_hub import notebook_login

notebook_login()

# 1. Download the MKQA datasets



In [6]:
lang = 'fr' #Set the language. Could be 'de' if german, 'fr' if french
language_data = data_preparation.prepare_dataset(lang)

#Print the length of the dataset and a sample query and answer
print(f"Length of {lang} dataset: {len(language_data['queries'])}, "
      f"sample {lang} query: {language_data['queries'][0]}, "
      f"sample {lang} answer: {', '.join(language_data['answers'][0])}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mkqa.py: 0.00B [00:00, ?B/s]

KeyboardInterrupt: Interrupted by user

# 2. Generate Answers and probabilities

Define input and output paths

In [ ]:
answers_file_path = f'data/generated_answers/output_answers_{lang}.jsonl'
span_file_path = f'data/outputs_partial_span_{lang}.jsonl'


In [ ]:
#Load the model and tokenizer from Huggingface
model_id = "meta-llama/Llama-3.1-8B-Instruct" #We used the Llama-3.1-8B-Instruct model
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

NameError: name 'language_data' is not defined

In [ ]:
#Generate the answers
data_preparation.generate_answers(model, tokenizer, language_data['queries'], lang, answers_file_path) #If you want to change the language, change the 'lang' at the code block above.

#3. Span mapping

In [ ]:
# file_path = f'data/outputs_partial_{lang}.jsonl'
# model_id = "meta-llama/Llama-3.1-8B-Instruct" #We used the Llama-3.1-8B-Instruct model
# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
# tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
# tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
data_preparation.span_mapping(answers_file_path, span_file_path, tokenizer)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs_partial_fr.jsonl'

#4. Convert soft labels to hard labels

In [ ]:
input_path = span_file_path
output_path = f'data/train_dataset_{lang}.jsonl'
data_preparation.generate_hard_labels(input_path, output_path)
train_data_path = output_path #Set the train_data_path

FileNotFoundError: [Errno 2] No such file or directory: 'outputs_partial_span_en.jsonl'

#5. Validation dataset reformat

In [ ]:
val_input_path = f'mushroom.{lang}-val.v2.extra.jsonl'
val_output_path = f'validation_dataset_{lang}.jsonl'
data_preparation.validation_reformat(val_input_path, val_output_path)
val_data_path = val_output_path #Set the val_data_path

#6. Model Training

In [ ]:
LABEL_LIST = ['O', 'B']
MODEL_NAME = 'FacebookAI/xlm-roberta-base'
data_files = {
        "train": train_data_path,
        "validation": val_data_path
    }
baseline_model.train_model(MODEL_NAME, LABEL_LIST, data_files, output_dir=f"./results_xmr_{lang}")

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.798500,3.516951,0.054571,1.000000,0.103495,0.054571
2,0.422600,4.150593,0.054571,1.000000,0.103495,0.054571
3,0.994000,4.704367,0.054571,1.000000,0.103495,0.054571


Training completed. Checkpoints saved to: ./results_xmr_en


#7. Model Testing

In [ ]:
test_path = f'mushroom.{lang}-tst.v1.extra.jsonl'
baseline_model.test_model(
    MODEL_NAME,
    test_lang=lang,
    model_path=f'results_xmr_{lang}/checkpoint-15000',
    data_path=test_path
)

Generating test split: 0 examples [00:00, ? examples/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Labels saved to en-hard_labels.json and en-soft_labels.json
Prediction file saved to en-pred.jsonl


#8. Evaluation

In [ ]:
reference = helpers.load_jsonl(test_path) #Change the filename
hard_references, soft_references = helpers.convert_reference_to_dict(reference)
predictions = helpers.load_jsonl(f"{lang}-pred.jsonl")

results = metrics.evaluate_predictions(predictions, hard_references, soft_references)

print("📊 Evaluation Results:")
print(f"Mean IoU: {results['mean_iou']:.4f}")
print(f"Mean Spearman: {results['mean_spearman']:.4f}")

📊 Evaluation Results:
Mean IoU: 0.4789
Mean Spearman: 0.2181
